# Sentiment analysis with Logistic Regression

## Logistic Regression

> Some of the explanations in this tutoarial are inspired by chapters 3 and 8 in the book _"Python Machine Learning"_ by Sebastian Raschka

First, let's introduce what Logistic Regression is. 

[Logistic Regression](https://en.wikipedia.org/wiki/Logistic_regression) is a classification model that is very easy to implement and performs very well on linearly separable classes. It is one of the most widely used
algorithms for classification in industry too, which makes it attractive to play with.

_Very_ simplistically explained, Logistic Regression works as follows:

![Logistic Regression](../data/misc/logistic_regression.png)

First we will define the input for our algorithm. The imput will be each sample in whatever dataset we are working with. Each sample will consist of several features. For example, if we're working with housing price prediction, the features for each sample could be the size of the house, number of rooms, etc. We'll call the input vector **X**.

For the algorythm to learn, we need to define variables that we can adjust accordingly to what we want to predict. We will create a vector of _weights_ (**W**) that the model will adjust in order to predict more accurately. The process of adjusting those weights is what we call **learning**.

For every input sample, we will perform a dot product of the features by the weights **XW**. This product is sometimes referred as _net input_. This will give us a real number. Since in this particular problem we want to _classify_ (positive/negative), we need squash this number in the range [0, 1]. This will give us the _probability_ of a positive event. A function that does precisely that is called **sigmoid**. The sigmoid function looks like this:

![sigmoid](../data/misc/sigmoid.svg)

What sigmoid is doing is basically transforming big inputs into a value close to 1, and small inputs into a value close to 0. This is exactly what we want. 

We will do this for every sample in our training set and compute the errors. To calculate the error we only need to compare our prediction with the true label for each sample. We will sum the square errors of all the samples to get a global prediction error. This will be our **cost function**.

A cost function is then something we want to minimize. **Gradient descent** is a method for finding the minimum of a function of multiple variables, such like the one we're dealing with here.

### Gradient descent

> Watch [this](https://www.youtube.com/watch?v=IHZwWFHWa-w) video for a visual introduction to Gradient Descent

Once all our training samples have been computed and the error calculated with our cost function, we need to _minimize_ that cost function. A method for doing that is gradient descent. There are many [articles](https://en.wikipedia.org/wiki/Gradient_descent) that contain detailed explanations _and_ implementations of GD, so let's not do this here. However is good to have an intuition.

For illustration purposes, let's think about a function with two parameters. Something like this one:

![Gradient Descent](../data/misc/Gradient_descent.png)

Gradient descent will try to find the minimum of the function. To do so, we calculate the slope of the function at a certain point, and move towards the direction that makes the function decrease. There are some things to have in mind though.

As you can see a function can have one or several _local minimum_. In a local minimum, the slope will be zero and GD will "think" it's found the global minimum. To avoid this, we can choose a bigger "step" when we move towards the minimum. The "size" of the step towards the minimun is what we call the **learning rate**, and it's another adjustable parameter.

We need to be careful here: If we choose a too small learning rate, we can get stuck in a local minimum. If we instead choose a too big learning rate, we risk overshooting the global minimum. We need to experiment, and the adecuate learning rate depends on the particular problem and the data.

### The training process

In order for logistic regression to learn, we need to repeat the process descrived before several times. Each one of these times is called an **epoch**. The number of epochs to run depends on the problem and the training data. It is... yes, another tunnable parameter of the algorithm.

The set of all tunnable parameters is called **hyperparameters** of the model.

Like with the leatning rate, we need to be careful when choosing the number of epochs: If we train too many epochs, we risk **overfitting**. This means that our model will "memorize" the training data and will generalize badly when presented new data. 

If we train too little, it will fail to find any pattern and the prediction accuracy will be very low. This is known as **underfitting**.

There are techniques that help prevent overfitting. These **regularization** techniques are out of the scope of this tutorial, but... guess! It's also something to tune and experiment with :)

This is why when training a model you need to set aside a _test dataset_ in order to know the accuracy of your algorithm in unknown data. The test dataset will **never** be used during training

## Sentiment analysis

Let's first of all have a look at the data. Load and split the data.

**NOTE: When reading the data, use `encoding='latin-1'`**

In [ ]:
import pandas as pd

# Load the data into a DataFrame
df = pd.read_csv('../data/twitter/train.csv', encoding='latin-1')
df.head()

In [ ]:
# Split the data into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X = df['SentimentText']
y = df['Sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
# How much data do you have in train and test?
n_samples_train = X_train.shape[0]
n_samples_test = X_test.shape[0]

print(f'There are {n_samples_train} samples in train and {n_samples_test} in test')

As we can see, the structure of a twit varies a lot between twit and twit. They have different lengths, letters, numbers, extrange characters, etc. 

It is also important to note that **a lot** of words are not correctly spelled, for example the word _"Juuuuuuuuuuuuuuuuussssst"_ or the word _"frie"_ instead of _"friend"_

This makes it hard to mesure how positive or negative are the words withing the corpus of twits. If they were all correct dictionary words, we could use a **lexicon** to punctuate words. However because of the nature of social media language, we cannot do that. 

So we need a way of scoring the words such that words that appear in positive twits have greater score that those that appear in negative twits.

But first... how do we represent the twits as vectors we can input to our algorithm?

### Bag of words

One thing we could do to represent the twits as equal-sized vectors of numbers is the following:

* Create a list (vocabulary) with all the unique words in the whole corpus of twits. 
* We construct a feature vector from each twit that contains the counts of how often each word occurs in the particular twit

_Note that since the unique words in each twit represent only a small subset of all the words in the bag-of-words vocabulary, the feature vectors will mostly consist of zeros_

Let's construct the bag of words. We will work with a smaller example for illustrative purposes, and at the end we will work with our real data.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

twits = [
    'This is amazing!',
    'ML is the best, yes it is',
    'I am not sure about how this is going to end...?'
]

count = CountVectorizer()
bag = count.fit_transform(twits)

count.vocabulary_

As we can see from executing the preceding command, the vocabulary is stored in a Python dictionary that maps the unique words to integer indices. Next, let's print the feature vectors that we just created:

In [ ]:
bag.toarray()

Each index position in the feature vectors corresponds to the integer values that are stored as dictionary items in the CountVectorizer vocabulary. For example, the first feature at index position 0 resembles the count of the word 'about' , which only occurs in the last document, and the word 'is' , at index position 7, occurs in all three twits (two times in the second twit). These values in the feature vectors are also called the **raw term frequencies**: `tf(t,d )` —the number of times a term `t` occurs in a document `d`.

### How relevant are words? Term frequency-inverse document frequency

We could use these raw term frequencies to score the words in our algorithm. There is a problem though: If a word is very frequent in _all_ documents, then it probably doesn't carry a lot of information. In order to tacke this problem we can use **term frequency-inverse document frequency**, which will reduce the score the more frequent the word is accross all twits. It is calculated like this:

\begin{equation*}
tf-idf(t,d) = tf(t,d) ~ idf(t,d)
\end{equation*}

_tf(t,d)_ is the raw term frequency descrived above. _idf(t,d)_ is the inverse document frequency, than can be calculated as follows:

\begin{equation*}
\log \frac{n_d}{1+df\left(d,t\right)}
\end{equation*}

where `n` is the total number of documents and _df(t,d)_ is the number of documents where the term `t` appears. 

The `1` addition in the denominator is just to avoid zero term for terms that appear in all documents. Ans the `log` ensures that low frequency term don't get too much weight.

Fortunately for us `scikit-learn` does all those calculations for us:

In [ ]:
import numpy as np

from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True,
                         norm='l2',
                         smooth_idf=True)

np.set_printoptions(precision=2)

# Feed the tf-idf transformer with our previously created Bag of Words
tfidf.fit_transform(bag).toarray()

As you can see, words that appear in all documents like _is_ (with 0.43 ), get a lower score than others that don't appear in all documents, like _amazing_ (with 0.72).

Note also that `norm='l2'` parameter: This is an important one, and what is doing is normalize the tf-idfs so that they're all in the same scale and thus work better with Logistic Regression.

Smooth idf weights by adding one to document frequencies, as if an extra document was seen containing every term in the collection exactly once. Prevents zero divisions.

## Data clean up (yay...)

### Removing stop words

Now that we know how to format and score our input, we can start doing the analysis! Can we?... Well, we _can_, but let's look at our **real** vocabulary. Specifically, the most common words:

In [ ]:
from collections import Counter

vocab = Counter()
for twit in train.SentimentText:
    for word in twit.split(' '):
        vocab[word] += 1

vocab.most_common(20)

As you can see, the most common words are meaningless in terms of sentiment: _I, to, the, and_... they don't give any information on positiveness or negativeness. They're basically **noise** that can most probably be eliminated. Let's see the whole distribution to convince ourselves of this:

In [ ]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

In [ ]:
import math

def plot_distribution(vocabulary):

    hist, edges = np.histogram(list(map(lambda x:math.log(x[1]),vocabulary.most_common())), density=True, bins=500)

    p = figure(tools="pan,wheel_zoom,reset,save",
               toolbar_location="above",
               title="Word distribution accross all twits")
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555", )
    show(p)

plot_distribution(vocab)

It's clear now that a porcion of the words are overly represented. These kind of words are called *stop words*, and it is a common practice to remove them when doing text analysis. Let's do it and see the distribution again:

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

vocab_reduced = Counter()
for w, c in vocab.items():
    if not w in stop:
        vocab_reduced[w]=c

vocab_reduced.most_common(20)

This looks better, only in the 20 most common words we already see words that make sense: _good, love, really_... Let's see the distribution now

In [ ]:
plot_distribution(vocab_reduced)

### Removing special characters and "trash"

We still se a very uneaven distribution. If you look closer, you'll see that we're also taking into consideration punctuation signs ('-', ',', etc) and other html tags like `&amp`. We can definitely remove them for the sentiment analysis, but we will try to keep the emoticons, since those _do_ have a sentiment load:

In [ ]:
import re

def preprocessor(text):
    """ Return a cleaned version of text
    """
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text

print(preprocessor('This!! twit yo :) is <b>nice</b>'))

### Stemming
We are almost ready! There is another trick we can use to reduce our vocabulary and consolidate words. If you think about it, words like: love, loving, etc. _Could_ express the same positivity. If that was the case, we would be  having two words in our vocabulary when we could have only one: lov. This process of reducing a word to its root is called **steaming**.

We also need a _tokenizer_ to break down our twits in individual words. We will implement two tokenizers, a regular one and one that does steaming:

In [ ]:
from nltk.stem import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):
    return text.split()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

print(tokenizer('Hi there, I am loving this, like with a lot of love'))
print(tokenizer_porter('Hi there, I am loving this, like with a lot of love'))

## Training Logistic Regression

We are finally ready to train our algorythm. We need to choose the best hyperparameters like the _learning rate_ or _regularization strength_. We also would like to know if our algorithm performs better steaming words or not, or removing html or not, etc...

To take these decisions methodically, we can use a Grid Search. Grid search is a method of training an algorythm with different variations of parameters to latter select the best combination

In [ ]:
from sklearn.model_selection import train_test_split

# split the dataset in train and test
X = train['SentimentText']
y = train['Sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

In the code line above, `stratify` will create a train set with the same class balance than the original set

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{
    'vect__stop_words': [stop, None],
    'vect__tokenizer': [tokenizer, tokenizer_porter],
    'vect__preprocessor': [None, preprocessor],
    'vect__use_idf':[False, True],
    'clf__penalty': ['l1', 'l2'],
    'clf__C': [1.0, 10.0, 100.0]
}]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=3,
                           n_jobs=-1)

In [ ]:
# Note: This may take a long while to execute, like... 1 or 2 hours
from sklearn.utils import parallel_backend

with parallel_backend('loky', n_jobs=-1):
    gs_lr_tfidf.fit(X_train, y_train)

**NOTE:** After running the cell above, I got these results

Interestingly, the set of parameters that best results give us are:

* A regularization strength of `1.0` using `l2` regularization
* Using our `preprocessor` (removing html, keeping emoticons, etc) _does_ improve the performance
* Surprisingly, removing stop words does not improve accuracy
* word steming doesn't seem to help either

As youcan see, sometimes intuition may lead to wrong decisions, and it's important to _test_ all our assumptions. 

Let's see what's our best accuracy then:

In [83]:
# Using the best parameters
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0, penalty='l2'))])
lr_tfidf.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=False, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling

In [86]:
predictions = lr_tfidf.predict(X_test)

In [87]:
from sklearn.metrics import accuracy_score

print(f'Accuracy: {accuracy_score(predictions, y_test):.3f}')

Accuracy: 0.767


If we would like to use the classifier in another place, or just not train it again and again everytime, we can save the model in a pickle file:

In [88]:
import pickle
import os

pickle.dump(lr_tfidf, open(os.path.join('../data/twitter', 'logisticRegression.pkl'), 'wb'), protocol=4)

Finally, let's run some tests :-)

In [89]:
twits = [
    "This is really bad, I don't like it at all",
    "I love this!",
    ":)",
    "I'm sad... :("
]
sent = {
    0: 'Sad',
    1: 'Happy'
}
preds = lr_tfidf.predict(twits)

for i in range(len(twits)):
    print(f'{twits[i]} --> {sent[preds[i]]}')

This is really bad, I don't like it at all --> Sad
I love this! --> Happy
:) --> Happy
I'm sad... :( --> Sad
